# Preliminaries

The following notebook contains code adapted from the Wrench tutorials (https://github.com/JieyuZ2/wrench.)

In [2]:
import wrench.wrench as wrnch
import spacy
import numpy as np
import nltk

import logging
import torch

from wrench.wrench.dataset import load_dataset
from wrench.wrench.logging import LoggingHandler
from wrench.wrench.endmodel import MLPModel
from wrench.wrench.labelmodel import MajorityVoting, FlyingSquid

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)

# Creating Labeling Functions

First, we need to load our data in to the Wrench-provided pipeline. A set of prior labeling features are provided in the dataset to use in lieu of user-provided labeling functions; in order to use them, set extract_feature to True.

In [20]:
# Set up the location from which to load the data.
dataset_home = './data'
data = 'youtube'
#### Extract data features using pre-trained BERT model and cache it
extract_fn = 'bert'
model_name = 'bert-base-cased'

#Note: you can set extract_features to True if you'd like to use pre-set Labeling Function outputs.
train_data, valid_data, test_data = load_dataset(dataset_home, data, extract_feature=True, extract_fn=extract_fn,
                                                 cache_name=extract_fn, model_name=model_name)

2022-03-15 12:48:57 - loading data from data/youtube/train.json


  0%|          | 0/1686 [00:00<?, ?it/s]

2022-03-15 12:48:57 - loading data from data/youtube/valid.json


  0%|          | 0/120 [00:00<?, ?it/s]

2022-03-15 12:48:57 - loading data from data/youtube/test.json


  0%|          | 0/250 [00:00<?, ?it/s]

2022-03-15 12:48:57 - loading features from data/youtube/train_bert.pkl
2022-03-15 12:48:57 - loading features from data/youtube/valid_bert.pkl
2022-03-15 12:48:57 - loading features from data/youtube/test_bert.pkl


Now, if you're going to manually generate and provide Labeling Functions, we're going to consider some ways of doing so:

In [13]:
#### Generate procedural labeling functions, using the LF generation defined by WRENCH.
from wrench.wrench.synthetic import ConditionalIndependentGenerator, NGramLFGenerator

#### Generate procedural labeling functions
generator = NGramLFGenerator(dataset=train_data, min_acc_gain=0.1, min_support=0.01, ngram_range=(1, 2))
applier = generator.generate(mode='correlated', n_lfs=10)
L_test = applier.apply(test_data)
L_train = applier.apply(train_data)

In [27]:
## Now, let's reinsert the newly labeled data back into the dataset.
# train_data, valid_data, test_data = load_dataset(dataset_home, data, extract_feature=True, extract_fn=applier.apply,
#                                                  )

#### Make sure you run a cell to generate the labeled data if you're providing manual functions!

# Wrench Training Pipeline

In [22]:
#### Generate soft training label via a label model
#### The weak labels provided by supervision sources are alreadly encoded in dataset object
label_model = MajorityVoting()
label_model.fit(dataset_train=train_data, dataset_valid=valid_data)
soft_label = label_model.predict_proba(labeled_train)

In [23]:
#### Train a MLP classifier with soft label
device = torch.device('cuda:0')
n_steps = 100000
batch_size = 128
test_batch_size = 1000 
patience = 200
evaluation_step = 50
target='acc'

model = MLPModel(n_steps=n_steps, batch_size=batch_size, test_batch_size=test_batch_size)

In [24]:
# Let's actually train the model here.
history = model.fit(dataset_train=train_data, dataset_valid=valid_data, y_train=soft_label, 
                    device=device, metric=target, patience=patience, evaluation_step=evaluation_step)

[TRAIN] MLP Classifier:   0%|                                                                                 …

2022-03-15 12:51:16 - [INFO] early stop @ step 30450!


In [26]:
#### Evaluate the trained model
metric_value = model.test(test_data, target)
print(metric_value)

0.84
